In [ ]:
%%capture
! pip install pandas numpy matplotlib sklearn tensorflow torchvision pytorch opencv-python

! export MallocStackLoggingNoCompact=1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import glob
import os

import librosa
import librosa.display


import torch
from torch import nn
from torchvision import models, transforms, datasets
from torchvision.models import ResNet18_Weights

from time import time
from tqdm import tqdm

In [ ]:
seed = 12
np.random.seed(seed)

# path_imgs = Path('./BinaryClassifiedSpectrograms/')
# path_imgs1 = Path('./PitchShiftedBinaryClassifiedSpectrograms/')
path_imgs = Path('./TempoShiftedBinaryClassifiedSpectrograms/')

batch_size = 32
# batch_size = 538

hop_length = 512

n_fft = 2048

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

In [ ]:
%%time

# Define Tranforms
train_transforms = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor(),
    
    # Resnet18_Weights.DEFAULT mean and std
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor(),

    # Resnet18_Weights.DEFAULT mean and std
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the data
# dataset = datasets.ImageFolder(path_imgs, transform = train_transforms)
train_dataset = datasets.ImageFolder(path_imgs, transform = train_transforms)
val_dataset = datasets.ImageFolder(path_imgs, transform = test_transforms)
test_dataset = datasets.ImageFolder(path_imgs, transform = test_transforms)


torch.manual_seed(1)
num_train_samples = len(train_dataset)
# num_train_samples = 5000

# Permute the data
indices = torch.randperm(num_train_samples)

# Split the data into Train and Validation
train_testval_split = 0.2
train_split = int(num_train_samples * train_testval_split)
val_split = int(train_split * 0.5)

train_subset = torch.utils.data.Subset(train_dataset, indices[train_split:])
val_subset = torch.utils.data.Subset(val_dataset, indices[val_split:train_split])
test_subset = torch.utils.data.Subset(test_dataset, indices[:val_split])

print(f"Length of Train:{len(train_subset)}; Length of Val:{len(val_subset)}; Length of Test:{len(test_subset)}")

# Make DataLoaders 
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_subset, 
    batch_size=batch_size,
    shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    dataset=val_subset,
    batch_size=batch_size,
    shuffle=True
)

# Classes
classes = train_dataloader.dataset.dataset.classes


In [ ]:
print(train_dataloader.dataset.dataset.classes)

unique, counts = np.unique(train_dataloader.dataset.dataset.targets, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
unique, counts = np.unique(train_dataloader.dataset.dataset.targets, return_counts=True)
print(dict(zip(unique, counts)))


In [ ]:
print(train_dataloader.dataset.dataset.classes)
print(train_dataloader.dataset.dataset.class_to_idx)

In [ ]:
(imgs, labels) = next(iter(train_dataloader))
print(imgs.shape)
print(labels.shape)
print(labels)


In [ ]:
debug = False
if debug:
    train_labels = set()
    for (img, label) in train_subset:
        train_labels.add(label)
    print(f"train_labels: {train_labels}")
    val_labels = set()
    for (img, label) in val_subset:
        val_labels.add(label)
    print(f"val_labels: {val_labels}")
    test_labels = set()
    for (img, label) in test_subset:
        test_labels.add(label)
    print(f"test_labels: {test_labels}")

In [ ]:
def mean_std(loader):
    images, lebels = next(iter(loader))
    # shape of images = [b,c,w,h]
    mean, std = images.mean([0,2,3]), images.std([0,2,3])
    return mean, std

mean, std = mean_std(train_dataloader)
print("mean and std: \n", mean, std)

In [ ]:
# Load a Pretrained Model
resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Fix the trainable parameters
for parameter in resnet.parameters():
    parameter.requires_grad = False
    
    
# Number of Input Features in the Last Fully Connected Layer
in_features = resnet.fc.in_features

# Replacing the Last Fully Connected Layer
fc = nn.Linear(in_features=in_features, out_features=len(classes))
# fc = nn.Linear(in_features=in_features, out_features=2)
resnet.fc = fc


# Updating the Weights and Bias of the last layer
params_to_update = []
for name, param in resnet.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

# Define the Loss and Optimizer Functions
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
def train(model, criterion, optimizer, train_dataloader, test_dataloader, print_every,num_epoch):
    
    steps = 0
    train_losses, val_losses = [], []
    
    model.to(device)
    for epoch in tqdm(range(num_epoch)):
        running_loss = 0
        correct_train = 0
        total_train = 0
        start_time = time()
        iter_time = time()
        
        model.train()
        for i, (images, labels) in enumerate(train_dataloader):
            steps += 1
            images = images.to(device)
            labels = labels.to(device)
            # labels = torch.nn.functional.one_hot(labels)

            # Forward pass
            output = model(images)
            loss = criterion(output, labels)

            correct_train += (torch.max(output, dim=1)[1] == labels).sum()
            total_train += labels.size(0)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            # Logging
            if steps % print_every == 0:
                print(f'Epoch [{epoch + 1}]/[{num_epoch}]. Batch [{i + 1}]/[{len(train_dataloader)}].', end=' ')
                print(f'Train loss {running_loss / steps:.3f}.', end=' ')
                print(f'Train acc {correct_train / total_train * 100:.3f}.', end=' ')
                with torch.no_grad():
                    model.eval()
                    correct_val, total_val = 0, 0
                    val_loss = 0
                    for images, labels in test_dataloader:
                        images = images.to(device)
                        labels = labels.to(device)
                        output = model(images)
                        loss = criterion(output, labels)
                        val_loss += loss.item()
                        
                        correct_val += (torch.max(output, dim=1)[1] == labels).sum()
                        total_val += labels.size(0)

                print(f'Val loss {val_loss / len(test_dataloader):.3f}. Val acc {correct_val / total_val * 100:.3f}.', end=' ')
                print(f'Took {time() - iter_time:.3f} seconds')
                iter_time = time()
                
                
                train_losses.append(running_loss / total_train)
                val_losses.append(val_loss / total_val)


        print(f'Epoch took {time() - start_time}') 
        torch.save(model, f'checkpoint_{correct_val / total_val * 100:.2f}')
        
    return model, train_losses, val_losses

In [ ]:
print_every = 25
num_epoch = 20

resnet, train_losses, val_losses = train(
    model=resnet,
    criterion=criterion,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,
    print_every=print_every,
    num_epoch=num_epoch
)


plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()


In [ ]:
pred_classes = ["deepfake", "human"]

y_test = []
y_pred = []
for img, label in test_subset:
    img = torch.Tensor(img)
    img = img.to(device)
    resnet.eval()
    prediction = resnet(img[None])
    
    final_pred = classes[torch.max(prediction, dim=1)[1]]
    # final_pred = torch.max(prediction, dim=1)[1]

    print(classes[label], final_pred)
    
    y_test.append(classes[label])
    y_pred.append(final_pred)

In [ ]:
print("Accuracy:",(100*(np.array(y_test) == np.array(y_pred)).sum()/len(y_test)))

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, pos_label="human")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred, labels=classes)#["deepfake", "human"])

tn, fp, fn, tp = cm.ravel()

#cmd = ConfusionMatrixDisplay(cm, display_labels=classes)#["deepfake", "human"])
#cmd.plot()
#plt.show()

print("True Negative: ", tn, end= '\t|\t')
print("False Positive: ", fp)
print("-" * 55)
print("False Negative: ", fn, end='\t|\t')
print("True Positive: ", tp)

In [ ]:
TPR = tp / (tp + fn)
FPR = fp / (fp + tn)
PPV = tp / (tp + fp)
Specificity = tn / (fp + tn)

print(TPR) # True Positive Rate / recall / sensitivity
print(FPR) # False Positive Rate / fall-out
print(PPV) # Positive Predictive Value / Precision
print(Specificity)